In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "X-Linked_Lymphoproliferative_Syndrome"
cohort = "GSE180393"

# Input paths
in_trait_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome"
in_cohort_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE180393"

# Output paths
out_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE180393.csv"
out_gene_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE180393.csv"
out_clinical_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE180393.csv"
json_path = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Analyzing the dataset for gene expression data
# From the background info, we see this is a microarray study on Affymetrix platform
# analyzing transcriptome data, which indicates gene expression data is available
is_gene_available = True

# 2. Variable availability and data type conversion
# 2.1 Data Availability
# Looking at the sample characteristics:
# For trait: The key 0 contains disease categories/sample groups
# Age: Not available in the characteristics
# Gender: Not available in the characteristics

trait_row = 0  # This corresponds to "sample group"
age_row = None  # Age data not available
gender_row = None  # Gender data not available

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert the trait value to binary: 
    1 for disease conditions, 0 for healthy controls (Living donor)
    """
    if value is None or ':' not in value:
        return None
    
    # Extract the value after the colon
    value = value.split(':', 1)[1].strip()
    
    # Determine trait status
    if value == "Living donor":
        return 0  # Healthy control
    else:
        return 1  # Disease condition (any type of kidney disease)

# Since age and gender data are not available, we define placeholder functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# The trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Validate and save the initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    try:
        # Extract clinical features using the provided clinical_data from previous step
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        print("Preview of selected clinical data:")
        print(preview_df(selected_clinical_df))
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error during clinical feature extraction: {e}")


### Step 3: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Reviewing the gene identifiers
# The identifiers like '100009613_at', '100009676_at', '10000_at' appear to be probe IDs from a microarray
# platform, likely Affymetrix, as indicated by the '_at' suffix.
# These are not standard human gene symbols and will need to be mapped to official gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns for gene mapping
probe_id_col = 'ID'
gene_id_col = 'ENTREZ_GENE_ID'

print(f"Using mapping from {probe_id_col} (probe identifiers) to {gene_id_col} (gene identifiers)")

# First, let's examine the format differences between gene_data and gene_annotation
print("\nSample probe IDs in gene expression data:")
print(gene_data.index[:5].tolist())
print("\nSample probe IDs in gene annotation:")
print(gene_annotation[probe_id_col][:5].tolist())

try:
    # Create a properly formatted mapping dictionary that will match the gene_data index
    mapping_dict = {}
    
    # Extract the base part of the probe IDs from gene_data (remove suffix if needed)
    for probe_id in gene_data.index:
        # Check if this probe exists directly in the annotation
        matching_rows = gene_annotation[gene_annotation[probe_id_col] == probe_id]
        
        if len(matching_rows) > 0:
            # Direct match found
            entrez_id = matching_rows.iloc[0][gene_id_col]
            mapping_dict[probe_id] = str(entrez_id)
        else:
            # Try matching without the "_at" suffix
            base_id = probe_id.split('_')[0] if '_' in probe_id else probe_id
            matching_rows = gene_annotation[gene_annotation[probe_id_col] == base_id]
            
            if len(matching_rows) > 0:
                entrez_id = matching_rows.iloc[0][gene_id_col]
                mapping_dict[probe_id] = str(entrez_id)
    
    print(f"\nCreated mapping for {len(mapping_dict)} probes")
    
    # Convert mapping_dict to DataFrame for apply_gene_mapping function
    mapping_df = pd.DataFrame({
        'ID': list(mapping_dict.keys()),
        'Gene': list(mapping_dict.values())
    })
    
    # Apply mapping to get gene expression data
    if len(mapping_df) > 0:
        # Skip the symbol extraction since we're using Entrez IDs directly
        # Create a custom function to apply the mapping
        def map_probes_to_genes(expression_df, mapping_df):
            """Map probes to genes using the mapping dataframe without symbol extraction"""
            # Add a sentinel column to track genes per probe (always 1 for this case)
            mapping_df['num_genes'] = 1
            mapping_df = mapping_df.set_index('ID')
            
            # Join expression data with mapping
            merged_df = mapping_df.join(expression_df, how='inner')
            
            # Get expression columns
            expr_cols = [col for col in merged_df.columns if col not in ['Gene', 'num_genes']]
            
            # Group by gene and sum expression values
            gene_expression_df = merged_df.groupby('Gene')[expr_cols].sum()
            
            return gene_expression_df
        
        # Apply custom mapping function
        gene_data = map_probes_to_genes(gene_data, mapping_df)
        
        print(f"\nAfter mapping, gene expression data shape: {gene_data.shape}")
        print("First 5 genes and 3 samples after mapping:")
        print(gene_data.iloc[:5, :3] if not gene_data.empty else "No genes mapped successfully")
        
        # Normalize gene symbols using the provided function if not empty
        if not gene_data.empty:
            print("\nNormalizing gene symbols...")
            try:
                # Since we're using Entrez IDs, we'll skip normalization
                # Save directly with Entrez IDs as gene identifiers
                os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
                gene_data.to_csv(out_gene_data_file)
                print(f"Gene expression data saved to {out_gene_data_file}")
            except Exception as e:
                print(f"Error during gene symbol normalization: {e}")
        else:
            print("No gene expression data to save after mapping.")
    else:
        print("No valid mappings found between probes and genes.")
        
except Exception as e:
    print(f"Error during gene mapping: {e}")
    import traceback
    traceback.print_exc()